In [ ]:
import random
import json

# Define classes
class Move:
    def __init__(self, name, damage, critical_hit_chance, accuracy, effect=None):
        self.name = name
        self.damage = damage
        self.critical_hit_chance = critical_hit_chance
        self.accuracy = accuracy
        self.effect = effect
        self.times_used = 0

    def __str__(self):
        return f"{self.name} - Damage: {self.damage}%, Crit: {self.critical_hit_chance}%, Acc: {self.accuracy}%"
    
    def to_dict(self):
        return {
            "name": self.name,
            "damage": self.damage,
            "critical_hit_chance": self.critical_hit_chance,
            "accuracy": self.accuracy,
            "effect": self.effect,
            "times_used": self.times_used
        }

class Pokemon:
    def __init__(self, name, type, health, moves=None):
        self.name = name
        self.type = type
        self.health = health
        self.max_health = health
        self.moves = moves if moves else []

    def add_move(self, move, player):
        if isinstance(move, Move):
            if "special" in move.name.lower() and move.times_used < 2:
                self.moves.append(move)
                player.inventory.append({'name': move.name, 'description': 'A special move'})
                print(f"Special move {move.name} added to inventory.")
            elif "special" not in move.name.lower():
                self.moves.append(move)
                player.inventory.append({'name': move.name, 'description': 'A regular move'})
                print(f"Move {move.name} added to inventory.")
            else:
                print(f"Cannot add move {move.name}: limit reached.")
        else:
            print("Invalid move type. Must be an instance of Move.")

    def is_defeated(self):
        return self.health <= 0

    def attack(self, other_pokemon, move_name, player):
        move = next((m for m in self.moves if m.name == move_name), None)
        if move and other_pokemon:
            if random.random() < move.accuracy / 100:
                damage = move.damage
                if random.random() < move.critical_hit_chance / 100:
                    damage *= 1.5
                    print(f"Critical hit by {self.name}!")
                other_pokemon.health -= damage
                move.times_used += 1
                print(f"{self.name} used {move_name} on {other_pokemon.name} causing {damage}% damage.")
                if other_pokemon.is_defeated():
                    player.record_win()
                if self.is_defeated():
                    player.record_loss()
            else:
                print(f"{self.name}'s attack missed!")
        else:
            print("Move not found or invalid target.")

    def to_dict(self):
        return {
            "name": self.name,
            "type": self.type,
            "health": self.health,
            "max_health": self.max_health,
            "moves": [move.to_dict() for move in self.moves]
        }

class Player:
    def __init__(self, name):
        self.name = name
        self.inventory = []
        self.team = []
        self.wins = 0
        self.losses = 0

    def to_dict(self):
        return {
            'name': self.name,
            'inventory': self.inventory,
            'team': [pokemon.to_dict() for pokemon in self.team],
            'wins': self.wins,
            'losses': self.losses
        }
    
    @classmethod
    def from_dict(cls, data):
        player = cls(data['name'])
        player.inventory = data['inventory']
        player.team = [Pokemon.from_dict(pokemon_data) for pokemon_data in data['team']]
        player.wins = data['wins']
        player.losses = data['losses']
        return player
    
    def add_item(self, item):
        self.inventory.append(item)

    def remove_item(self, item):
        self.inventory.remove(item)

    def display_inventory(self):
        print(f"{self.name}'s Inventory:")
        for item in self.inventory:
            item_details = f"{item['name']} - {item['description']}"
            if 'effectiveness' in item:
                item_details += f" - Effectiveness: {item['effectiveness']}"
            if 'heal' in item:
                item_details += f" - Heal: {item['heal']}"
            print(item_details)

    def save_inventory(self):
        with open(f"{self.name}_inventory.json", "w") as file:
            json.dump(self.inventory, file)

    def load_inventory(self):
        try:
            with open(f"{self.name}_inventory.json", "r") as file:
                self.inventory = json.load(file)
                print(f"{self.name}'s inventory loaded successfully!")
        except FileNotFoundError:
            print("No inventory found.")

    def add_pokemon(self, pokemon):
        self.team.append(pokemon)
        self.inventory.append({'name': pokemon.name, 'description': f'{pokemon.name}, a {pokemon.type} type Pokémon'})
        print(f"{pokemon.name} added to team and inventory.")

    def remove_pokemon(self, pokemon):
        self.team.remove(pokemon)

    def display_team(self):
        print(f"{self.name}'s Team:")
        for pokemon in self.team:
            print(f"{pokemon.name} - HP: {pokemon.health}/{pokemon.max_health}")

    def record_win(self):
        self.wins += 1
        print(f"{self.name} has won! Total wins: {self.wins}")

    def record_loss(self):
        self.losses += 1
        print(f"{self.name} has lost! Total losses: {self.losses}")

def select_avatar_and_moves():
    # Define moves
    aqua_jet = Move("Aqua Jet", 35, 20, 80)
    fire_breath = Move("Fire Breath", 40, 14, 85)
    leaf_storm = Move("Leaf Storm", 38, 12, 75)
    hydro_pump = Move("Hydro Pump", 45, 10, 70)
    thunder_shock = Move("Thunder Shock", 30, 15, 90)
    rock_throw = Move("Rock Throw", 25, 20, 80)
    solar_beam = Move("Solar Beam", 55, 25, 60)
    dragon_rage = Move("Dragon Rage", 40, 5, 100)
    psychic = Move("Psychic", 35, 10, 90)
    tackle = Move("Tackle", 15, 5, 95)
    gust = Move("Gust", 20, 10, 100)
    ember = Move("Ember", 30, 12, 90)
    vine_whip = Move("Vine Whip", 30, 10, 85)
    water_gun = Move("Water Gun", 25, 15, 90)
    peck = Move("Peck", 20, 10, 100)
    scratch = Move("Scratch", 20, 5, 95)
    razor_leaf = Move("Razor Leaf", 35, 15, 80)
    thunder_wave = Move("Thunder Wave", 10, 20, 100)
    flamethrower = Move("Flamethrower", 50, 10, 70)
    ice_beam = Move("Ice Beam", 50, 15, 75)
    poison_sting = Move("Poison Sting", 15, 5, 100)
    bubble = Move("Bubble", 20, 10, 100)
    bite = Move("Bite", 25, 15, 90)
    quick_attack = Move("Quick Attack", 25, 20, 95)
    confusion = Move("Confusion", 30, 10, 90)
    absorb = Move("Absorb", 20, 10, 100)
    thunderbolt = Move("Thunderbolt", 40, 20, 80)
    poison_jab = Move("Poison Jab", 35, 10, 80)
    mud_slap = Move("Mud Slap", 20, 15, 85)
    horn_attack = Move("Horn Attack", 35, 15, 90)
    wing_attack = Move("Wing Attack", 35, 20, 85)

    # Define available Pokemons with their moves
    available_pokemons = [
        Pokemon("Squirtle", "Water", 75, [aqua_jet, water_gun, bubble, tackle]),
        Pokemon("Charmander", "Fire", 75, [fire_breath, ember, flamethrower, scratch]),
        Pokemon("Bulbasaur", "Grass", 75, [leaf_storm, vine_whip, razor_leaf, tackle]),
        Pokemon("Pidgey", "Flying", 60, [gust, quick_attack, wing_attack, peck]),
        Pokemon("Ratata", "Normal", 55, [tackle, bite, scratch, quick_attack])
    ]
    # Selecting avatar (Pokemon)
    print("Choose your Pokemon:")
    for i, pokemon in enumerate(available_pokemons, 1):
        print(f"{i}. {pokemon.name}")

    while True:
        try:
            choice = int(input("Enter the number of your choice: "))
            if 1 <= choice <= len(available_pokemons):
                selected_pokemon = available_pokemons[choice - 1]
                print(f"You selected {selected_pokemon.name}!")
                break
            else:
                print("Invalid choice. Please enter a number within the range.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    # Selecting moves for the chosen Pokemon
    selected_moves = []
    print(f"Select moves for {selected_pokemon.name}:")
    for i, move in enumerate(selected_pokemon.moves, 1):
        print(f"{i}. {move.name}")

    while len(selected_moves) < 2:
        try:
            choice = int(input(f"Enter the number of move {len(selected_moves) + 1}: "))
            if 1 <= choice <= len(selected_pokemon.moves):
                selected_moves.append(selected_pokemon.moves[choice - 1])
                print(f"Move {selected_moves[-1].name} added!")
            else:
                print("Invalid choice. Please enter a number within the range.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    return selected_pokemon, selected_moves

def select_opponent():
    possible_opponents = [charmander, bulbasaur, pidgey, ratata]
    return random.choice(possible_opponents)

def save_game(player):
    player_data = {
        'name': player.name,
        'inventory': player.inventory,
        'team': [pokemon.to_dict() for pokemon in player.team],
        'wins': player.wins,
        'losses': player.losses
    }
    with open(f"{player.name}_data.json", "w") as file:
        json.dump(player_data, file)
    print(f"Game saved for player {player.name}!")


def load_game(player_name):
    try:
        with open(f"{player_name}_data.json", "r") as file:
            player_data = json.load(file)
            player = Player(player_data["name"])
            player.inventory = player_data["inventory"]
            player.team = [Pokemon(pokemon_data["name"], 
                                   pokemon_data["type"], 
                                   pokemon_data["health"], 
                                   [Move(move["name"], move["damage"], 
                                         move["critical_hit_chance"], move["accuracy"]) 
                                    for move in pokemon_data["moves"]]) 
                           for pokemon_data in player_data["team"]]
            player.wins = player_data["wins"]
            player.losses = player_data["losses"]
            print(f"Game loaded successfully for {player.name}!")
            return player
    except FileNotFoundError:
        print(f"No saved game found for {player_name}. Starting a new game.")
        return None
    except json.JSONDecodeError:
        print(f"Malformed JSON data in {player_name}_data.json. Starting a new game.")
        return None


# Define opponent Pokemon
charmander = Pokemon("Charmander", "Fire", 75, [Move("Fire Breath", 40, 14, 85)])
bulbasaur = Pokemon("Bulbasaur", "Grass", 75, [Move("Leaf Storm", 38, 12, 75)])
pidgey = Pokemon("Pidgey", "Flying", 60, [Move("Gust", 20, 10, 100)])
ratata = Pokemon("Ratata", "Normal", 55, [Move("Tackle", 15, 5, 95)])

# Game loop
def main():
    print("Welcome to Pokemon Battle Game!")
    player_name = input("Enter your player name: ")
    player = load_game(player_name)  # Pass the player_name to the function
    if player is None:
        player = Player(player_name)
        selected_pokemon, selected_moves = select_avatar_and_moves()
        selected_pokemon.moves = selected_moves
        player.add_pokemon(selected_pokemon)
    else:
        print(f"Welcome back, {player.name}!")
  
    while True:
        print("\nWhat would you like to do?")
        print("1. View Inventory")
        print("2. View Team")
        print("3. Attack with Pokemon")
        print("4. Add Player with Moves")
        print("5. Remove Player from Team")
        print("6. Save Game")
        print("7. Quit")
        choice = input("Enter your choice: ")

        if choice == "1":
            player.display_inventory()
        elif choice == "2":
            player.display_team()
        elif choice == "3":
            attacking_pokemon = player.team[0]
            opponent_pokemon = select_opponent()
            move_choice = random.choice(attacking_pokemon.moves).name
            attacking_pokemon.attack(opponent_pokemon, move_choice, player)
        elif choice == "4":
            player_name = input("Enter the name of the new player: ")
            selected_pokemon, selected_moves = select_avatar_and_moves()
            selected_pokemon.moves = selected_moves
            new_player = Player(player_name)
            new_player.add_pokemon(selected_pokemon)
            player.add_pokemon(new_player)
            print(f"{player_name} added to the team!")
        elif choice == "5":
            player_name = input("Enter the name of the player to remove: ")
            for pokemon in player.team:
                if pokemon.name == player_name:
                    player.remove_pokemon(pokemon)
                    print(f"{player_name} removed from the team!")
                    break
            else:
                print(f"{player_name} not found in the team.")
        elif choice == "6":
            save_game(player)
            player.save_inventory()  # Save inventory when saving game
        elif choice == "7":
            print("Thanks for playing!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


Welcome to Pokemon Battle Game!


Enter your player name:  jojo


No saved game found for jojo. Starting a new game.
Choose your Pokemon:
1. Squirtle
2. Charmander
3. Bulbasaur
4. Pidgey
5. Ratata


Enter the number of your choice:  1


You selected Squirtle!
Select moves for Squirtle:
1. Aqua Jet
2. Water Gun
3. Bubble
4. Tackle


...

Squirtle used Water Gun on Pidgey causing 25% damage.

What would you like to do?
1. View Inventory
2. View Team
3. Attack with Pokemon
4. Add Player with Moves
5. Remove Player from Team
6. Save Game
7. Quit


Enter your choice:  1


jojo's Inventory:
Squirtle - Squirtle, a Water type Pokémon

What would you like to do?
1. View Inventory
2. View Team
3. Attack with Pokemon
4. Add Player with Moves
5. Remove Player from Team
6. Save Game
7. Quit


Enter your choice:  6


Game saved for player jojo!

What would you like to do?
1. View Inventory
2. View Team
3. Attack with Pokemon
4. Add Player with Moves
5. Remove Player from Team
6. Save Game
7. Quit
